In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA
# Load the pre-trained ResNet50 model without including the top fully connected layers.
model = ResNet50(weights='imagenet', include_top=False)

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    return img_data


root_dir = r'D:\HE_dataset'
features = []
labels = []
label_names = []
filename = []

for staining_folder in os.listdir(root_dir):
    staining_path = os.path.join(root_dir, staining_folder)
    img_dir = os.path.join(staining_path, 'img')
    if os.path.isdir(img_dir):
        label_names.append(staining_folder)
        for img_file in os.listdir(img_dir):
            if img_file.endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
                img_path = os.path.join(img_dir, img_file)
                img_data = load_and_preprocess_image(img_path)
                feature = model.predict(img_data)
                feature = feature.flatten()
                features.append(feature)
                labels.append(staining_folder)
                filename.append(img_file)

features = np.array(features)


# t-SNE 
tsne = TSNE(n_components=2, random_state=0)
tsne_result = tsne.fit_transform(features)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import ConvexHull
import numpy as np

label_color_mapping = {
    '10×DAPI': (0.1216, 0.4667, 0.7059, 1.0),  # tab10 蓝色
    'DAPI': (0.1725, 0.6275, 0.1725, 1.0),  # tab10 绿色
    'HE': (0.8392, 0.1529, 0.1569, 1.0),  # tab10 红色
    '10×HE': (1.0, 0.4980, 0.0549, 1.0),  # tab10 橙色
    'mIF': (0.5804, 0.4039, 0.7412, 1.0),  # tab10 紫色
    'ssDNA': (0.8902, 0.4667, 0.7608, 1.0),  # tab10 粉色
}


plt.rcParams.update({'font.family': 'Arial', 'font.size': 20})

plt.figure(figsize=(10, 8))  

assigned_colors = {}

for label in label_color_mapping:
    indices = [j for j, l in enumerate(labels) if l == label]
    color = label_color_mapping[label]
    alpha_value = 1.0 if label == 'CellBinDB' else 0.6
    zorder_value = 4 if label == 'CellBinDB' else 2
    
    plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], 
                label=label, c=[color], marker='o', s=5, alpha=alpha_value, zorder=zorder_value)

handles, labels_ = plt.gca().get_legend_handles_labels()

plt.title('t-SNE of Image Features')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.legend(handles, labels_, loc='upper left', bbox_to_anchor=(1.05, 1.05), borderaxespad=0., fontsize=16) 
plt.tight_layout()  


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.savefig(r'', dpi=600, bbox_inches='tight') 
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import ConvexHull
import numpy as np

label_color_mapping = {
    'CellBinDB': (0.8392, 0.1529, 0.1569, 1.0),  # tab10 红色
    'bbbc038': (0.1216, 0.4667, 0.7059, 1.0),  # tab10 蓝色
    'IEEE_TMI_2019': (1.0, 0.4980, 0.0549, 1.0),  # tab10 橙色
    'Lizard': (0.5804, 0.4039, 0.7412, 1.0),  # tab10 紫色
    'TissueNet': (0.1725, 0.6275, 0.1725, 1.0),  # tab10 绿色
    'kromp': (0.8902, 0.4667, 0.7608, 1.0),  # tab10 粉色
    'MoNuSeg': (0.4980, 0.4980, 0.4980, 1.0),  # tab10 灰色
    'NuInsSeg': (0.0902, 0.7451, 0.8118, 1.0),  # tab10 蓝绿色
}

plt.rcParams.update({'font.family': 'Arial', 'font.size': 20})
plt.figure(figsize=(10, 8)) 

assigned_colors = {}

for label in label_color_mapping:
    indices = [j for j, l in enumerate(labels) if l == label]
    color = label_color_mapping[label]
    alpha_value = 1.0 if label == 'CellBinDB' else 0.6
    zorder_value = 4 if label == 'CellBinDB' else 2
    
    plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], 
                label=label, c=[color], marker='o', s=5, alpha=alpha_value, zorder=zorder_value)
    assigned_colors[label] = color


cellbindb_indices = [j for j, l in enumerate(labels) if l == 'CellBinDB']
cellbindb_points = tsne_result[cellbindb_indices, :2]


hull = ConvexHull(cellbindb_points)
hull_points = cellbindb_points[hull.vertices]

# Use polygon path to draw the convex hull area.
polygon = plt.Polygon(hull_points, color=(0.8392, 0.1529, 0.1569, 0.1), edgecolor='red', linewidth=2, zorder=1)
plt.gca().add_patch(polygon)


handles, labels_ = plt.gca().get_legend_handles_labels()
our_index = labels_.index('CellBinDB')
handles.insert(0, handles.pop(our_index))
labels_.insert(0, labels_.pop(our_index))

plt.title('t-SNE of Image Features')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.legend(handles, labels_, loc='upper left', bbox_to_anchor=(1.05, 1.05), borderaxespad=0., fontsize=16) 
plt.tight_layout()  


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.savefig(r'', dpi=600, bbox_inches='tight') 
plt.show()
